# Imports

In [2]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, AutoModelForTokenClassification, pipeline
import textstat
import huspacy
import spacy

import re
import nltk
import string
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from rouge import Rouge 
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

nltk.download('punkt')
nltk.download('stopwords')

# Models

In [59]:
tokenizer = AutoTokenizer.from_pretrained("SZTAKI-HLT/mT5-base-HunSum-1")
model = AutoModelForSeq2SeqLM.from_pretrained("SZTAKI-HLT/mT5-base-HunSum-1")

In [110]:
tokenizer = AutoTokenizer.from_pretrained("SZTAKI-HLT/Bert2Bert-HunSum-1")
model = AutoModelForSeq2SeqLM.from_pretrained("SZTAKI-HLT/Bert2Bert-HunSum-1")

# Control Case

In [5]:
def run_control_case(prompt, model, tokenizer):
    print(f"Model: {model.config.to_dict()['_name_or_path']}")
    print("----------------------------------------------")

    input_ids = tokenizer.encode(prompt, return_tensors="pt")
    output = model.generate(
        input_ids,
        num_beams=10,
        no_repeat_ngram_size=2,
        encoder_no_repeat_ngram_size=3,
        length_penalty=2,
        max_length=128,
        early_stopping=True
    )
    
    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
    print("Output: ", generated_text)

# Force Word Tests

In [109]:
def run_force_word_test_case(test_case, prompt, force_words, model, tokenizer):
    print(f"Test Case: {test_case}")
    print(f"Model: {model.config.to_dict()['_name_or_path']}")
    print("----------------------------------------------")
    
    input_ids = tokenizer.encode(prompt, return_tensors="pt")
    force_tokens = []
    for force_word in force_words:
        token = tokenizer.encode(force_word, add_special_tokens=False)
        force_tokens.append(token)
        
    output = model.generate(
        input_ids,
        force_words_ids=force_tokens,
        num_beams=10,
        no_repeat_ngram_size=2,
        encoder_no_repeat_ngram_size=3,
        length_penalty=2,
        max_length=128,
        early_stopping=True
    )
    
    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
    print("Output: ", generated_text)
    print()
    print("forced all force_words_ids:", all(word in generated_text for word in force_words))

## Test Case 1: Text already has the force_word

In [30]:
prompt = """
Nyilvánosan bocsánatot kért az HBO és a Max vezérigazgatója és elnöke, Casey Bloys azután, hogy kiderült, hamis Twitter-fiókokat használt arra, hogy HBO-sorozatokról szóló negatív kritikák alatt kommenteljen, írja a Variety.

Bloys bocsánatkérésében arról beszélt, hogy bár tudja, „hülye ötlet volt”, az motiválta, hogy nagyon szenvedélyesen szereti a csatornája műsorait, illetve „azokat az embereket, akik dolgoznak rajtuk”. A vezérigazgató arról beszélt, csak azt akarja, hogy „a műsorok nagyszerűek legyenek, az emberek szeressék őket”.
„Nagyon fontos számomra, hogy mit gondolnak a műsorokról. Erre gondoltam akkor is, amikor 2020–2021-ben otthonról dolgoztam és egészségtelenül sokat lapozgattam a Twittert. Aztán támadt egy hülye ötletem, hogy kiadjam a frusztrációmat” – összegezte Bloys. Bocsánatot kért a kritikusoktól, majd beszélt még arról is, hogy hat fiókot kezelt másfél éven át. Bloys bocsánatkérése egy nappal azután hangzott el, hogy a Rolling Stone közzétett egy cikket, amely részletezi az ügyvezető és az HBO elleni indított pert – ezt a vállalat egy korábbi alkalmazottja, Sumy Temori indította el, aki úgy érzi, jogtalanul rúgták ki őt.

A lap ehhez vett elő 2020-as és 2021-es üzeneteket, amelyekben Bloys többször is beszélt arról, hogy kamu Twitter-fiókokkal kéne válaszolni a negatívan író kritikusoknak. Temori akkoriban ügyvezetői asszisztens volt, állította, hogy utasították ezeknek a profiloknak az elkészítésére.
"""
force_words = ["HBO"]
test_case = "Text already has the force_word"

In [34]:
run_control_case(prompt, model, tokenizer)

Model: SZTAKI-HLT/Bert2Bert-HunSum-1
----------------------------------------------
Output:  A HBO vezérigazgatója elismerte, hogy igazak voltak azok a vádak, amiket az HBO - t ért negatív kommentek miatt indított.


In [35]:
run_force_word_test_case(test_case, prompt, force_words, model, tokenizer)

Test Case: Text already has the force_word
Model: SZTAKI-HLT/Bert2Bert-HunSum-1
----------------------------------------------
Output:  A HBO vezérigazgatója elismerte, hogy igazak voltak azok a vádak, amiket az HBO - t ért negatív kommentek miatt indított.

forced all force_words_ids: True


## Test Case 2: Text already has multiple force_words

In [40]:
prompt = """
Orbán Viktor magyar miniszterelnök csütörtökön kijelentette, hogy büszke Vlagyimir Putyin orosz elnökkel való kapcsolataira, akivel ebben a hónapban Kínában találkozott mindannak ellenére, hogy az Európai Unió kitartó erőfeszítéseket tesz Moszkva elszigetelésére az Ukrajna elleni háború miatt.

Orbán Viktor az EU 27 tagállamának vezetőit tömörítő brüsszeli csúcstalálkozóra érkezve beszélt erről. A magyar kormánynak nagyon világos és átlátható a stratégiája, ami eltér a többség, „valószínűleg az önök stratégiájától is” – mondta a miniszterelnök egy újságírói kérdésre. „Mindent szeretnénk megtenni a béke érdekében. Ezért minden kommunikációs vonalat nyitva tartunk az oroszok felé, különben nem lesz esély a békére. Ez egy stratégia, ezért büszkék vagyunk rá.

Mi vagyunk az egyetlenek, akik a béke nevében és érdekében beszélünk, ami mindenkinek érdeke lenne Európában”.

Költségvetési emelés: hiányoznak a „politikai érvek”
A magyar miniszterelnököt megkérdezték róla, mi a véleménye a többéves uniós költségvetés emeléséről. (Orbán Viktor korábban ezt elfogadhatatlannak nevezte, ebben a cikkünkben írtunk róla, mire kér az EU több pénzt és hol sántítanak a magyar kormány ellenérvei.) „Több pénzt akarnak beszedni a tagállamoktól, köztük tőlünk is, hogy odaadhassák migráció… befogadáspolitikára” és „az ukránoknak” – mondta a miniszterelnök. Az Európai Bizottság ugyanolyan, a migrációt visszafogó megállapodásokra kért többletforrásokat, mint amilyet Tunéziával kötött és Egyiptommal tervez, de Orbán Viktor arról beszélt, „nem akarunk pénzt adni a migránsoknak”.
"""
force_words = ["Vlagyimir", "Putyin"]
test_case = "Text already has the force_word"

In [41]:
run_control_case(prompt, model, tokenizer)

Model: SZTAKI-HLT/mT5-base-HunSum-1
----------------------------------------------
Output:  A miniszterelnök Brüsszelben beszélt az EU-csúcs előtti utolsó beszédéről, és arról is, mit gondol a költségvetés emeléséről.


In [42]:
run_force_word_test_case(test_case, prompt, force_words, model, tokenizer)

Test Case: Text already has the force_word
Model: SZTAKI-HLT/mT5-base-HunSum-1
----------------------------------------------
Output:  A miniszterelnök Putyinnal való kapcsolatairól beszélt Brüsszelben, ahol az EU-csúcstalálkozón elmondta, büszke rá, és büszkén fogadja a háborús oroszokat. A költségvetés emelése viszont elfogadhatatlan, mert hiányoznak az európai politikai ervek. Az uniós tagállamok vezetőinek csúcstalálkozóján úgy fogalmazott, nem akarnak pénzt adni az ukrán állampolgároknak, de ígéretet tett arra, többet akar adni az unió tőle Vlagyimir

forced all force_words_ids: True


## Test Case 3: Proper Nouns

In [43]:
prompt = """
Orbán Viktor magyar miniszterelnök csütörtökön kijelentette, hogy büszke Vlagyimir Putyin orosz elnökkel való kapcsolataira, akivel ebben a hónapban Kínában találkozott mindannak ellenére, hogy az Európai Unió kitartó erőfeszítéseket tesz Moszkva elszigetelésére az Ukrajna elleni háború miatt.

Orbán Viktor az EU 27 tagállamának vezetőit tömörítő brüsszeli csúcstalálkozóra érkezve beszélt erről. A magyar kormánynak nagyon világos és átlátható a stratégiája, ami eltér a többség, „valószínűleg az önök stratégiájától is” – mondta a miniszterelnök egy újságírói kérdésre. „Mindent szeretnénk megtenni a béke érdekében. Ezért minden kommunikációs vonalat nyitva tartunk az oroszok felé, különben nem lesz esély a békére. Ez egy stratégia, ezért büszkék vagyunk rá.

Mi vagyunk az egyetlenek, akik a béke nevében és érdekében beszélünk, ami mindenkinek érdeke lenne Európában”.

Költségvetési emelés: hiányoznak a „politikai érvek”
A magyar miniszterelnököt megkérdezték róla, mi a véleménye a többéves uniós költségvetés emeléséről. (Orbán Viktor korábban ezt elfogadhatatlannak nevezte, ebben a cikkünkben írtunk róla, mire kér az EU több pénzt és hol sántítanak a magyar kormány ellenérvei.) „Több pénzt akarnak beszedni a tagállamoktól, köztük tőlünk is, hogy odaadhassák migráció… befogadáspolitikára” és „az ukránoknak” – mondta a miniszterelnök. Az Európai Bizottság ugyanolyan, a migrációt visszafogó megállapodásokra kért többletforrásokat, mint amilyet Tunéziával kötött és Egyiptommal tervez, de Orbán Viktor arról beszélt, „nem akarunk pénzt adni a migránsoknak”.
"""
force_words = ["Vlagyimir Putyin"]
test_case = "Text already has the force_word"

In [47]:
run_control_case(prompt, model, tokenizer)

Model: SZTAKI-HLT/Bert2Bert-HunSum-1
----------------------------------------------
Output:  A Viktor - Putyin csúcson a magyar miniszterelnök azt mondta, az EU - nak nem szabad beleavatkoznia az ukrajnai konfliktusba, mert az a béke mellett szól.


In [48]:
run_force_word_test_case(test_case, prompt, force_words, model, tokenizer)

Test Case: Text already has the force_word
Model: SZTAKI-HLT/Bert2Bert-HunSum-1
----------------------------------------------
Output:  A Viktor - Vlagyimir Putyin csúcstalálkozón a magyar miniszterelnök azt mondta, az EU - nak nem szabad beleavatkoznia az ukrajnai konfliktusba, mert az a béke és a biztonság politikája. Az EU 27 tagországának vezetõit tömörítõ Európai Unió 27 tagállamainak vezetőit kérdeztük.

forced all force_words_ids: True


## Test Case 4: Common Words with Multiple Meanings

In [49]:
prompt = """
Azért járt rendezvényről rendezvényre autóval Karácsony Gergely főpolgármester az autómentes hétvégén, mert egy térdsérülés miatt nem tudott biciklizni – válaszolta a Telex kérdéseire a Főpolgármesteri Hivatal Sajtókapcsolati Csoportja.

Hétfőn a Bors írt arról, hogy Karácsony mindenhová autóval ment a hétvégén, sőt azt írták, hogy a sofőrje még szabálytalanul is parkolt. Emellett a különböző rendezvényeken pont a kerékpározást és a tömegközlekedést népszerűsítette, még villamosokkal is fotózkodott.

A cikk miatt arról kérdeztük a Főpolgármesteri hivatalt, hogy valóban autózott-e, és arról, hogy valóban szabálytalanul parkolt-e.

„A főpolgármester a hétvégén öt különböző fővárosi programon vett részt. A bicikli használatát egy térdsérülés akadályozta. Ellentétben a propagandamédia állításával a szolgálati autó sofőrje nem parkolt a járdán, hanem egy társasház kapubeállójában állt meg a kiszállás idejére szabályosan”

– áll a hivatal válaszában.
Arra is rákérdeztünk, hogy a főpolgármester tudott-e volna tömegközlekedési eszközzel menni egyik rendezvényről a másikra, de erre nem kaptunk választ.
"""
force_words = ["Karácsony"]
test_case = "Common Words with Multiple Meanings"

In [53]:
run_control_case(prompt, model, tokenizer)

Model: SZTAKI-HLT/mT5-base-HunSum-1
----------------------------------------------
Output:  A Fidesz viszont nem zsarol vagy követel – egyebek mellett erről beszélt a 24.hu-nak adott interjúban Schmitt Pál. A Hit Gyülekezetének vezetője coming outolt.


In [54]:
run_force_word_test_case(test_case, prompt, force_words, model, tokenizer)

Test Case: Common Words with Multiple Meanings
Model: SZTAKI-HLT/mT5-base-HunSum-1
----------------------------------------------
Output:  A főpolgármester nem tudta megakadályozni a biciklihasználatot, ezért ment autóval rendezvényekről rendezvényre. A Telexnek adott válaszában úgy nyilatkozott, Karácsony nem volt hajlandó tömegközlekedéssel közlekedni, és nem is tudott kerékpározni.

forced all force_words_ids: True


In [111]:
prompt = """
Emberölés bűntettének kísérlete miatt emelt vádat a Veszprém Megyei Főügyészség azzal a 36 éves férfival szemben, aki egy évvel ezelőtt karácsony este megkéselte a barátját Sümegen – derül ki az ügyészség közleményéből.

A vádirat szerint a két barát rendszeresen együtt ivott a helyi kocsmában, 2021. december 24-én viszont nem oda, hanem egyikük házába mentek, ahol iszogattak, zenét hallgattak és beszélgettek. Este tíz óra körül már mindketten ittasak voltak, amikor az egyik férfi kiment a fürdőszobába, majd visszafelé a konyhaasztalról felvett egy 13 centiméter pengehosszúságú kést, a szobába érve pedig minden előzetes ok nélkül mellkason szúrta a barátját, akinek mindössze annyit mondott, hogy

„Ne félj, nem fog fájni!”

A barát a szúrás erejétől a mögötte lévő ágyra esett, és eszméletét vesztette, a férfi pedig távozott.

Amikor a mellkason szúrt férfi magához tért, megpróbálta felhívni a testvérét, aki nem vette fel. Mivel állapota rosszabbodott, éjjel két óra körül egy videót küldött magáról a testvérének, és megírta neki, hogy megkéselték. A testvér ezt már észlelte, értesítette a rendőrséget és a mentőket, akik az életveszélyes állapotban lévő sértettet kórházba szállították. A férfi életét végül a szakszerű orvosi beavatkozás mentette meg.

Az ügyészség a vádiratában az elkövetővel szemben börtönbüntetés kiszabását és a közügyek gyakorlásától történő eltiltását indítványozta azzal, hogy az elkészítő ülésen történő beismerése esetén a Veszprémi Törvényszék kilenc év börtönbüntetésre és kilenc év közügyek gyakorlásától eltiltásra ítélje. Az ügyészség annak megállapítását is indítványozta, hogy bűnössége esetén a vádlott a törvény rendelkezésénél fogva nem bocsátható feltételes szabadságra.
"""
force_words = ["karácsony"]
test_case = "Common Words with Multiple Meanings"

In [112]:
run_control_case(prompt, model, tokenizer)

Model: SZTAKI-HLT/Bert2Bert-HunSum-1
----------------------------------------------
Output:  A barát egy évvel ezelõtt veszett össze a testvérével, majd a veszekedés közben egy késsel a mellkasába szúrt.


In [113]:
run_force_word_test_case(test_case, prompt, force_words, model, tokenizer)

Test Case: Common Words with Multiple Meanings
Model: SZTAKI-HLT/Bert2Bert-HunSum-1
----------------------------------------------
Output:  A testvér nem vette észre, hogy a férfi nem iszik. Az áldozat életét a mentõk mentették meg, az ügyészség vádat emeltek ellene, kilenc év börtönt indítványoz az ügyész. A fiú életét azonban nem mentette fel a bíróság, a testvért pedig örökre eltiltották foglalkozása gyakorlásától, így a vádlottnak kilenc évet kell rácsok mögött töltenie, ha nem kap gyors orvosi beavatkozást, mint ahogy tette azt egy másik vádlottal szemben is - írja a police. hu a Veszprém megyei ügyészségre hivatkozva. Ittas, veszekedõ barátját késelte meg egy férfi, aki az eset karácsony

forced all force_words_ids: True


## Test Case 5: Emotional Language

In [122]:
prompt = """
Azt hihetnénk, hogy a válságok, tragédiák idején jobban összefogunk embertársainkkal és empatikusabban viselkedünk a másikkal. A tapasztalat viszont nem egészen ezt mutatja. Míg sokan vannak azok, akik valóban elkezdenek segíteni – az orosz-ukrán háború kitörésekor is rengetegen kezdetek adománygyűjtésbe, ajánlottak fel szállást vagy egyéb módon segítettek és segítenek azóta is –, addig csaknem ugyanennyinek tűnik azoknak a száma is, akik mintha csak erre a szikrára vártak volna, hogy frusztrációjukat az online térben kiadva magukból tovább szítsák a feszültséget, és megértés helyett inkább az agressziót, támadást és veszekedést választják.

Traumáink fogságában
„Szerintem érdemes több komponenst figyelembe venni. Ha van egy csoport, amely fenyegetve érzi magát, valamint krízisbe kerül – hiszen nem lehet kikerülni a problémát, a korábbi megoldások pedig nem tűnnek elégségesnek –, akkor elindul egyfajta regresszív (a regresszió a fejlődés során már túlhaladott fázisok vagy stádiumok újbóli megjelenése – a szerk.) folyamat. Ilyenkor nem konstruktív, felnőtt módon állunk egy helyzethez – magyarázza Lázár Gergely pszichológus. – Számtalan bizonytalan információval találkozunk. Mikor ér véget a háború? Ki a felelős érte? Érinteni fog-e bennünket? Ha érint, akkor emberéletet is követelhet az országtól vagy csupán gazdasági következményekkel kell számolnunk? És így tovább. Mindenki válaszokat akar. Ez normális.

Az agresszióban tehát ott van a fenyegettség érzése, a törekvés arra, hogy valahogy kontrollálni próbáljuk az eseményeket, meg akarjuk védeni magunkat, és ez paradox módon kicsiben – például az online térben – harcot generál.

A másik ilyen az általános elvárásainkból indul ki. Ha egy nagyon intelligens ember például érzelmi szituációkban kritikán alul viselkedik, az feltűnik. Ha egy alapvetően összetartást, empátiát igénylő környezetben valaki agresszív, az is. Tehát nem feltétlenül biztos, hogy »sokkal agresszívebbek lennénk az online térben, mint máskor«, de valószínű, hogy jobban szemet szúr számunkra ez a viselkedés.”
"""
force_words = ["agresszió"]
test_case = "Emotional Language"

In [125]:
run_control_case(prompt, model, tokenizer)

Model: SZTAKI-HLT/mT5-base-HunSum-1
----------------------------------------------
Output:  Egyre több olyan ember van Magyarországon, aki a közösségi médiában, az interneten keresztül próbálja elviselni az emberek feszültségét, fenyegetését és támadását. A pszichológust kérdeztük arról, miként lehet hatékonyan kezelni az agresszív viselkedést, és milyen következményei lehetnek annak, ha valaki úgy érzi, nincs rá szükség.


In [126]:
run_force_word_test_case(test_case, prompt, force_words, model, tokenizer)

Test Case: Emotional Language
Model: SZTAKI-HLT/mT5-base-HunSum-1
----------------------------------------------
Output:  A háborúk idején a legtöbben úgy érzik magukat, mintha nem is lenne olyan könnyű dolguk. A helyzet akkor alakulhat ki, ha az emberek nem csak saját magukkal, hanem az egész világgal is szembesülnek. Az agresszív viselkedés azonban sokkal nagyobb veszélyt jelenthet az életünkre, és ezzel együtt növelheti az aggodalmukat is. Lázár György pszichológust kérdeztük arról, milyen eszközökkel lehet hatékonyan megelőzni agresszió

forced all force_words_ids: True


## Test Case 6: Idiomatic Expressions

In [133]:
prompt = """
Narendra Modi indiai miniszterelnök a Nemzetközi Olimpiai Bizottság (NOB) 141. ülésszakának megnyitó ünnepségén Mumbaiban megerősítette országa elkötelezettségét a 2036-os olimpia megrendezése mellett, és kijelentette, hogy "minden követ megmozgatnak" ennek érdekében.

Az olimpiai játékok megrendezése régi vágya Indiáénak, a miniszterelnök bizakodását fejezte ki, hogy a NOB támogatását is elnyeri ezekhez a törekvésekhez.

A KÖVETKEZŐ NYÁRI JÁTÉKOKAT A TERVEK SZERINT 2024-BEN PÁRIZSBAN, MAJD 2028-BAN LOS ANGELESBEN RENDEZIK MEG, A 2032-ES OLIMPIA RENDEZŐ VÁROSÁNAK PEDIG AZ AUSZTRÁLIAI BRISBANE-T VÁLASZTOTTÁK.

A NOB nyilatkozata szerint több érdeklődő is van a későbbi játékokra, és Indiát is nagy esélyesnek látják. A dél-ázsiai országnak tett gesztus, hogy a 2028-as olimpiai játékok programjába beveszi a krikettet, amely különösen Indiában örvend óriási népszerűségnek.
"""
force_words = ["minden követ megmozgat"]
test_case = "Idiomatic Expressions"

In [134]:
run_control_case(prompt, model, tokenizer)

Model: SZTAKI-HLT/Bert2Bert-HunSum-1
----------------------------------------------
Output:  A indiai kormány hivatalosan is bejelentette, hogy India támogatja az olimpiai játékok megpályázását - írja a The Daily Telegraph


In [135]:
run_force_word_test_case(test_case, prompt, force_words, model, tokenizer)

Test Case: Idiomatic Expressions
Model: SZTAKI-HLT/Bert2Bert-HunSum-1
----------------------------------------------
Output:  A indiai kormány bejelentette, hogy India támogatja a 2026 - os olimpiát, és a Nemzetközi Olimpia Bizottság is támogatja az eseményt. A Nemzetközi Olimpiai Iroda közleménye szerint a dél - koreai kormány támogatja Indiát, de nem minden esetben támogatja azt. Az olimpia megrendezését a 2040 - es olimpiára beveszi India, így a kriktik is nagy eséllyel pályáznak a játékok megrendezésére. India a jelek szerint nem áll le, folytatja a felkészülést a következő ötkarikás játékokra. Eközben az indiai olimpiai játékok szervezõi is bejelentették : a NDK - n kívül más országokkal is tárgyal minden követ megmozgat

forced all force_words_ids: True


## Test Case 7: Historical Context

In [153]:
prompt = """
Ma van a trianoni békeszerződés 102. évfordulója, ez a nap 2010. óta Magyarországon a nemzeti összetartozás napja is egyben. Az évfordulóról a pártok és politikusok is megemlékeztek.

Fidesz – A kormánypárt Orbán Viktor soraival emlékezett meg az összetartozás napjáról: „Van egy nagy nemzet, amely messze túlnyúlik az országhatárokon. Ennek a lelki közösségnek a megőrzése olyan nehéz száz év után, mint amit megéltünk, mégiscsak sikerült. A nemzeti összetartozás gondolata, érzésvilága erős, talán erősebb, mint a korábbi évtizedekben bármikor is volt.”

KDNP – Közleményükben azt írják, hogy „történelmünk során sokszor váltunk nagyhatalmak játékszerévé, de mindig ragaszkodtunk függetlenségünkhöz, nyelvünkhöz, hitünkhöz, kultúránkhoz, tradícióinkhoz, amelyek biztosították nemzetünk fennmaradását a legnehezebb korszakokban is.” A párt kitért arra is, hogy szerintük a baloldali-liberális politika el akarta magától taszítani a határon túli magyarságot. Továbbá kiemelik, hogy Magyarország segítségnyújtását Kárpátaljának: „Az anyaország mindig segítő kezet nyújt a bajbajutottaknak, most különösképpen a kárpátaljai testvéreinknek”

Karácsony Gergely – „Trianon közös tragédiája a magyar nemzetnek, kitörölhetetlen. Számomra elsősorban édesanyám története, aki a második bécsi döntéssel visszacsatolt területen született” – írja le személyes történetét a főpolgármester. Felhívta a figyelmet, hogy Trianon kérdésében nincsenek oldalak, csak közös gyász. „Jó volna, hogy ha végre megtanulnánk a ”vagy„ helyett ”és„-t használni, és a nemzeti összetartozás napján megfogadni, Trianonból tanulva, hogy magyarokként egyek vagyunk.”
"""
force_words = ["Trianont"]
test_case = "Historical Context"

In [156]:
run_control_case(prompt, model, tokenizer)

Model: SZTAKI-HLT/Bert2Bert-HunSum-1
----------------------------------------------
Output:  A Fidesz, a KDNP és a Fidesz is megemlékezést tart ma.


In [157]:
run_force_word_test_case(test_case, prompt, force_words, model, tokenizer)

Test Case: Historical Context
Model: SZTAKI-HLT/Bert2Bert-HunSum-1
----------------------------------------------
Output:  A Fidesz – KDNP és a Fidesz is megemlékezett a 102 évvel ezelõtti Trianon évfordulójáról. A kormánypártok és az ellenzék is emlékezik, a Jobbik pedig megemlékezést tart. Az Origo összegyűjtötte, mit tartanak fontosnak a pártok, és mit gondolnak a nemzet összetartozásáról a mai megemlékezéseken. Cikkünk a Trianont ünneplõ pártok megemlékezéseit foglalja össze.

forced all force_words_ids: True


## Test Case 8: Geographic Feature

In [158]:
prompt = """
A Magyar Kutatási Hálózat (Hungarian Research Network, HUN-REN), a Debreceni Egyetem (DE), az Eötvös Loránd Tudományegyetem (ELTE) és a Hortobágyi Nemzeti Park Igazgatóság kutatói drónokkal vizsgálták a hortobágyi Przewalski-ló ménes szociális rendszerét.

A kutatók két drón segítségével nagy idő- és térbeli felbontással követték a 278 tagú ménes minden egyedének mozgását, miközben az állatok többségét egyedileg azonosították. Az eredmények szerint a vadlovak az emberekhez hasonlóan összetett, többszörösen tagozódott társadalomban élnek, amelynek csoportszerkezete, sőt nemrég bekövetkezett és a jövőben várható változásai is megérthetők ezzel a módszerrel. A kutatásról szóló tanulmány kedden jelent meg a Nature Communications című tudományos folyóiratban – közölte a Magyar Kutatási Hálózat és az ELTE.

Egy nagy állatcsoport szociális viselkedésének vizsgálata klasszikus megfigyelési módszerekkel fáradságos, időigényes munka. A kutatók tanulmánya azonban rávilágít arra, hogy nagy felbontású adatokat gyűjtve az állatok mozgásáról, már néhány percnyi felvétel is elegendő információt szolgáltathat ahhoz, hogy a populáció társas felépítését megismerjék a szakemberek, és akár a csoport dinamikájának múltjáról és jövőjéről is következtetéseket vonhassanak le.

„A hortobágyi Przewalski-ménes csoportos mozgását szerettük volna vizsgálni. Azonban közel 300 állat egyidejű megfigyelése korántsem egyszerű feladat” – magyarázza a közleményben Ozogány Katalin, a tanulmány első szerzője, a HUN-REN DE Viselkedésökológiai Kutatócsoport munkatársa.

Az állatvilágban ritka az a többszintű társadalmi struktúra, amely az emberekre is jellemző. Főleg főemlősöknél találkozhatunk vele, de ceteknél, elefántoknál és egyes patásoknál is előfordul, hogy az egyedek kisebb, főként családi csoportokat alkotnak (például háremeket vagy egy matriarcha által vezetett rokoni csoportot), és ezek a családi csoportok egy nagyobb, lazább közösséget alkotnak.

A Hortobágyon 1997 óta élnek Przewalski-lovak a Pentezug rezervátumban. Az alapítás utáni első években a vadlovak háremei egymástól távol, saját területen mozogtak, és egymással ritkán kerültek interakcióba. Az utóbbi években azonban a háremek együtt egy nagy ménest alkotnak, amelyben a háremek továbbra is elkülöníthetők, de a rezervátum területén együtt mozognak. A kutatók ennek az összetett, háremekből álló ménesnek a csoportos mozgását vizsgálták.
"""
force_words = ["Hortobágy"]
test_case = "Geographic Feature"

In [162]:
run_control_case(prompt, model, tokenizer)

Model: SZTAKI-HLT/mT5-base-HunSum-1
----------------------------------------------
Output:  Több mint 300 különböző fajt figyeltek meg drónok segítségével a magyar-horvát határ közelében élő vándorló lovasok, akik az emberhez hasonlóan összetettebb társadalomba tartoznak.


In [163]:
run_force_word_test_case(test_case, prompt, force_words, model, tokenizer)

Test Case: Geographic Feature
Model: SZTAKI-HLT/mT5-base-HunSum-1
----------------------------------------------
Output:  A ló egyedei olyan összetett társadalmi csoportot alkotnak az emberhez hasonlóan, amelyek a legtöbbszörösen tagozódó társadalomba tartoznak – állapította meg egy nemzetközi kutatócsoport kutatása. A legújabb kutatások szerint az állatvilág egyik legritkább csoportja, melynek tagjait eddig nem sikerült azonosítani. Az emberi evolúciós folyamatok miatt azonban egyre nagyobb problémát jelent az élőlények csoportos viselkedése, és az utóbbi idő Hortobágy

forced all force_words_ids: True


## Test Case 9: Introducing a Related Concept

In [185]:
prompt = """
Klónozni tudja a podcasterek hangját a Spotify, így a műsorokat pillanatok alatt más nyelvekre tudják fordítani – írja a The Verge.

A ChatGPT-t gyártó OpenAI és a Spotify együttműködéséből született fejlesztéssel a podcasterek automatikusan létrehozhatják a műsoraik idegen nyelvű változatait, ráadásul mindezt a saját hangjukon. A Spotify már partnerségre is lépett néhány ismert amerikai podcasterrel, többek között Dax Sheparddal, Monica Padmannel, Lex Fridmannel, Bill Simmonsszal és Steven Bartlettel, hogy az angol nyelvű adásaikat spanyolra fordítsák le. A következő hetekben a francia és német fordítások bevezetését is tervezik.

A fejlesztés alapja az OpenAI nyílt forráskódú beszédfelismerő rendszere, a Whisper, ami képes az angol beszéd átírására és más nyelvekre fordítására. A Spotify új eszköze azonban túlmutat egy sima fordítóprogramon, ugyanis a fordítás a podcasterek szintetizált hangján szólal meg.

„Azáltal, hogy a Voice Translation a készítő saját hangját adja vissza, a hallgatók világszerte minden eddiginél hitelesebb módon fedezhetik fel és inspirálódhatnak a podcasterektől” – mondta Ziad Sultan, a Spotify személyre szabásért felelős alelnöke.
"""
force_words = ["mesterséges intelligencia"]
test_case = "Inferring and Introducing a Related Concept"

In [192]:
run_control_case(prompt, model, tokenizer)

Model: SZTAKI-HLT/Bert2Bert-HunSum-1
----------------------------------------------
Output:  A Spatify olyan új technológiát fejlesztett ki, amely képes idegen nyelven is angol nyelvre fordítani az adásokat. Az OpenAI - nak köszönhetõen a hangminõsége sokkal egyedibb, mint az angolé.


In [193]:
run_force_word_test_case(test_case, prompt, force_words, model, tokenizer)

Test Case: Inferring and Introducing a Related Concept
Model: SZTAKI-HLT/Bert2Bert-HunSum-1
----------------------------------------------
Output:  A Spatify olyan új technológiát fejlesztett ki, amely képes idegen nyelvre fordítani az adásokat. A hangfelismerõ rendszer a hangon szólal meg, így sokkal gyorsabban és sokkal kevesebb nyelvi fordulatra képes fordítani a mûsorokat. Az OpenAI - nak köszönhetõen a po postasterjátékok gyorsabban képesek megtanulni idegen nyelven is a saját hangjaikat, ráadásul még a nyelvtan is gyorsabban elsajátítható. Így a csatorna is hitelesebb és hitelesebbé válhat. Egy új fejlesztésnek hála azonban az új technológia nem csak az angol, hanem a magyar hangokra is lefordítja a beszélgetéseket, méghozzá olyan nyelven, mesterséges intelligencia

forced all force_words_ids: True


## Test Case 10: False information

In [202]:
prompt = """
Klónozni tudja a podcasterek hangját a Spotify, így a műsorokat pillanatok alatt más nyelvekre tudják fordítani – írja a The Verge.

A ChatGPT-t gyártó OpenAI és a Spotify együttműködéséből született fejlesztéssel a podcasterek automatikusan létrehozhatják a műsoraik idegen nyelvű változatait, ráadásul mindezt a saját hangjukon. A Spotify már partnerségre is lépett néhány ismert amerikai podcasterrel, többek között Dax Sheparddal, Monica Padmannel, Lex Fridmannel, Bill Simmonsszal és Steven Bartlettel, hogy az angol nyelvű adásaikat spanyolra fordítsák le. A következő hetekben a francia és német fordítások bevezetését is tervezik.

A fejlesztés alapja az OpenAI nyílt forráskódú beszédfelismerő rendszere, a Whisper, ami képes az angol beszéd átírására és más nyelvekre fordítására. A Spotify új eszköze azonban túlmutat egy sima fordítóprogramon, ugyanis a fordítás a podcasterek szintetizált hangján szólal meg.

„Azáltal, hogy a Voice Translation a készítő saját hangját adja vissza, a hallgatók világszerte minden eddiginél hitelesebb módon fedezhetik fel és inspirálódhatnak a podcasterektől” – mondta Ziad Sultan, a Spotify személyre szabásért felelős alelnöke.
"""
force_words = ["iTunes", "Meta"]
test_case = "False information"

In [203]:
run_control_case(prompt, model, tokenizer)

Model: SZTAKI-HLT/mT5-base-HunSum-1
----------------------------------------------
Output:  Automatikusan lefordíthatja a hangját az amerikai és francia podcastereknek szánt új fejlesztés, amelynek köszönhetően az angol műsorokat más nyelveken is le lehet fordítani.


In [204]:
run_force_word_test_case(test_case, prompt, force_words, model, tokenizer)

Test Case: False information
Model: SZTAKI-HLT/mT5-base-HunSum-1
----------------------------------------------
Output:  A chatGPT készítői a hangjukat fordítják át más nyelvre, ezzel pedig az adásokat szinte teljesen más verziók formájában tudják lefordítani. Az OpenAI fejlesztésének köszönhetően azonban már csak úgy tudnak fordítást készíteni, mint egy egyszerű fordítóprogrammal, de az új technológia sokkal hitelesebbé teszi az angol és francia nyelvű műsorok terjedését. A podcasterek most olyan eszközöket fejlesztettek ki, amelyek képesek az átírásra és Meta iTunes

forced all force_words_ids: True


## Test Case 11: Irrelevant Information

In [209]:
prompt = """
Kongatják a vészharangot az Európai Bizottság (EB) politikai tanácsadói, miután a személygépkocsik új emissziós követelményeit meghatározó Euro
7-es motorszabvány erőtlen és késedelmes változata áll nyerésre. Az Európai Parlament plenáris ülése e hét csütörtökön, november 9-én szavazhat a rendeletről.

A körvonalazódó alku alapját az a tavaly novemberi EB-javaslat jelenti, amely szigorú emissziós normákat írt volna elő a belső égésű motoros járművekre, függetlenül a felhasznált üzemanyagtól. A jelenlegi kibocsátási határértékek a személy- és kisteherautókra (Euro 6b), valamint a buszokra, teherautókra és egyéb nehézgépjárművekre (Euro VI) vonatkoznak.

A most az asztalra került Euro 7-es javaslat olyan új szabályokat is tartalmaz, mint a gumiabroncsokból származó mikroműanyagokra és a fékekből származó részecskékre, valamint az akkumulátorok tartósságára vonatkozó követelmények. A VoxEurop független médiaportál elemzése szerint azonban az utóbbiak csak arra szolgálnak, hogy eltereljék a figyelmet az eredetileg tervezett kibocsátási szabályok enyhítéséről. Az EB által létrehozott, Konzorcium a Rendkívül Alacsony Járműkibocsátásért nevű tanácsadó testület (CLOVE) által a VoxEurop számára készített elemzésen alapuló becslés szerint évi 4 milliárd eurós egészségügyi és környezeti kárt lehetne elkerülni azzal, ha a gyártók a profitjuk töredékéről lemondanának annak érdekében, hogy tisztább modelleket gyártsanak.
"""
force_words = ["karalábé"]
test_case = "Irrelevant Information"

In [210]:
run_control_case(prompt, model, tokenizer)

Model: SZTAKI-HLT/Bert2Bert-HunSum-1
----------------------------------------------
Output:  motorok Európai Parlamenti szakpolitikusai szerint az Euro 7 károsanyag - kibocsátási normák megszegése évente 4 millió eurós egészségügyi kárt okozna.


In [211]:
run_force_word_test_case(test_case, prompt, force_words, model, tokenizer)

Test Case: Irrelevant Information
Model: SZTAKI-HLT/Bert2Bert-HunSum-1
----------------------------------------------
Output:  Az Európai Bizottság politikai tanácsadója, az Euro 7 károsanyag - kibocsátási normákat kidolgozó Consorcium szerint az EU - ban évente 4 milliárd euró egészségügyi kárt lehetne megelőzni azzal, hogy a gyártók lemondanák tisztább modelljeik gyártásáról. A tervezetet az Európai Parlament politikai tanácsa terjeszti el, de az EP - képviselők csak arra szánnak figyelmet, hogyan lehetne enyhíteni az emissziósségi normák enyhítésének enyhítésén, írja a Világgazdaság. Az új szabályozást az euro 7 - rõl szóló uniós rendeletre alapozzák. Egy elemzõ szerint évi négy karalábé

forced all force_words_ids: True


# Omit Word Tests

In [2]:
def run_omit_word_test_case(test_case, prompt, bad_words, model, tokenizer):
    print(f"Test Case: {test_case}")
    print(f"Model: {model.config.to_dict()['_name_or_path']}")
    print("----------------------------------------------")
    
    input_ids = tokenizer.encode(prompt, return_tensors="pt")
    bad_tokens = []
    for bad_word in bad_words:
        token = tokenizer.encode(bad_word, add_special_tokens=False)
        bad_tokens.append(token)
    output = model.generate(
        input_ids,
        bad_words_ids=bad_tokens,
        num_beams=10,
        no_repeat_ngram_size=2,
        encoder_no_repeat_ngram_size=3,
        length_penalty=2,
        min_length=30,
        max_length=128,
        early_stopping=True
    )

    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
    print("Output: ", generated_text)
    print()
    print("omitted all bad_words_ids:", all(word not in generated_text for word in bad_words))

## Test Case 1: Negation

In [83]:
prompt = """
Novembertől a nagyközönség számára is elérhetővé vált a Credential Manager nevezetű funkció, amelynek köszönhetően a korábbinál sokkal biztonságosabb platformmá válhat az Android – legalábbis a Google szerint.
Ahogy arról korábban már az Index is beszámolt, az idén októberben megtartott Google I/O-n rengeteg mindenről szó esett. A Google amellett, hogy bemutatta legújabb telefonjait, a Google Pixel 8-at és a Pixel 8 Prót, illetve új okosóráját, a Google Pixel Watch 2-t, egyéb témákra is kitért – például a biztonságra.
"""
bad_words = ["Google"]
test_case = "Negation"

In [69]:
run_control_case(prompt, model, tokenizer)

Model: SZTAKI-HLT/Bert2Bert-HunSum-1
----------------------------------------------
Output:  A Google szerint sokkal biztonságosabbá válik az Android, mint azt eddig gondolták.


In [84]:
run_omit_word_test_case(test_case, prompt, bad_words, model, tokenizer)

Test Case: Negation
Model: SZTAKI-HLT/mT5-base-HunSum-1
----------------------------------------------
Output:  Az Android biztonságosabbá válhat, mint az eddigi platformok – így vélekedett a keresőóriás. A funkciót novemberben jelentették be.

omitted all bad_words_ids: True


## Test Case 2: Multiple Negation

In [85]:
prompt = """
Novembertől a nagyközönség számára is elérhetővé vált a Credential Manager nevezetű funkció, amelynek köszönhetően a korábbinál sokkal biztonságosabb platformmá válhat az Android – legalábbis a Google szerint.
Ahogy arról korábban már az Index is beszámolt, az idén októberben megtartott Google I/O-n rengeteg mindenről szó esett. A Google amellett, hogy bemutatta legújabb telefonjait, a Google Pixel 8-at és a Pixel 8 Prót, illetve új okosóráját, a Google Pixel Watch 2-t, egyéb témákra is kitért – például a biztonságra.
"""
bad_words = ["Google", "biztonságosabbá"]
test_case = "Negation"

In [77]:
run_control_case(prompt, model, tokenizer)

Model: SZTAKI-HLT/Bert2Bert-HunSum-1
----------------------------------------------
Output:  A Google szerint sokkal biztonságosabbá válik az Android, mint azt eddig gondolták.


In [86]:
run_omit_word_test_case(test_case, prompt, bad_words, model, tokenizer)

Test Case: Negation
Model: SZTAKI-HLT/mT5-base-HunSum-1
----------------------------------------------
Output:  Többek között arról is beszélt a keresőóriás, milyen biztonsági funkciót kapott az Android, és hogy mennyire biztonságos az operációs rendszer.

omitted all bad_words_ids: True


## Test Case 3: Negation Noun

In [119]:
prompt = """
„Amit önök javasolnak, az a magyar emberek érdeke, a magyar túszok ellen lett volna. (...) A külpolitika bonyolult terület. Akkor merészkedjenek oda, hogyha értik, hogy mit beszélnek.”

Orbán Viktor szokatlanul indulatosan szállt bele Toroczkai Lászlóba hétfőn a parlamentben, miután a Mi Hazánk elnöke és frakcióvezetője azt kérdezte tőle, hogy Magyarország miért nem támogatta az ENSZ gázai tűzszünetre felszólító határozatát.

Hétfőn a miniszterelnök bent volt a parlamentben, a legjobb pillanatokat ebben a videóban gyűjtöttük össze. Orbán és Toroczkai vitája 4:46-nál kezdődik.
"""
bad_words = ["Orbán Viktor", "Toroczkait"]
test_case = "Negation Noun"

In [128]:
run_control_case(prompt, model, tokenizer)

Model: SZTAKI-HLT/Bert2Bert-HunSum-1
----------------------------------------------
Output:  A nem kaptak rá engedélyt, a vendéglősnél viszont megjelentek az ellenőrök.


In [129]:
run_omit_word_test_case(test_case, prompt, bad_words, model, tokenizer)

Test Case: Idiomatic Expressions
Model: SZTAKI-HLT/Bert2Bert-HunSum-1
----------------------------------------------
Output:  A nem kaptak rá engedélyt, a vendéglősnél viszont megjelentek az ellenőrök, mondván, hogy nem minõsül bûncselekménynek az attak.

omitted all bad_words_ids: True


## Test Case 4: Idiomatic Expressions

In [130]:
prompt = """
Régi keletű a felismerés, miszerint az idő pénz. Akad bank, amelynek ügyfelei csak annyit foglalkoznak pénzügyeikkel, amennyit feltétlenül kell, hogy idejüket az igazán fontos dolgoknak szentelhessék.
Bár a hivatalos statisztikák szerint a bizonytalan piaci helyzet és a borús kilátások okozzák a legkínzóbb főfájást a hazai kis- és középvállalkozásoknak (kkv-knak), a személyes ügyféltárgyalásokon a konkrét nehézségek között az időhiány az egyik leggyakrabban visszatérő motívum.
"""
bad_words = ["idő", "pénz"]
test_case = "Idiomatic Expressions"

In [131]:
run_control_case(prompt, model, tokenizer)

Model: SZTAKI-HLT/Bert2Bert-HunSum-1
----------------------------------------------
Output:  A nem kaptak rá engedélyt, a vendéglősnél viszont megjelentek az ellenőrök.


In [132]:
run_omit_word_test_case(test_case, prompt, bad_words, model, tokenizer)

Test Case: Idiomatic Expressions
Model: SZTAKI-HLT/Bert2Bert-HunSum-1
----------------------------------------------
Output:  A nem kaptak rá engedélyt, a vendéglősnél viszont megjelentek az ellenőrök, mondván, hogy nem minõsül bûncselekménynek az attak.

omitted all bad_words_ids: True


# Named Entity Recognition

In [78]:
ner_tokenizer = AutoTokenizer.from_pretrained("akdeniz27/bert-base-hungarian-cased-ner")
ner_model = AutoModelForTokenClassification.from_pretrained("akdeniz27/bert-base-hungarian-cased-ner")

In [127]:
def run_ner_test_case(test_case, prompt, model, tokenizer, ner_model, ner_tokenizer, threshold=0.99):
    print(f"Test Case: {test_case}")
    print(f"Model: {model.config.to_dict()['_name_or_path']}")
    print("----------------------------------------------")
    
    ner = pipeline('ner', model=ner_model, tokenizer=ner_tokenizer, aggregation_strategy="first")
    entities = ner(prompt)
    
    named_entities = []

    for entity in entities:
        if entity["score"] >= threshold and entity["entity_group"] == "PER":
            named_entities.append(entity["word"])
    named_entities = set(named_entities)
    
    input_ids = tokenizer.encode(prompt, return_tensors="pt")
    force_tokens = []
    for force_word in named_entities:
        token = tokenizer.encode(force_word, add_special_tokens=False)
        force_tokens.append(token)
                
    entities_present = False
    output = model.generate(
        input_ids,
        force_words_ids=force_tokens,
        num_beams=10,
        no_repeat_ngram_size=2,
        encoder_no_repeat_ngram_size=3,
        length_penalty=2,
        max_length=128,
        early_stopping=True
    )
            
    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
    entities_present = all(entity in generated_text for entity in named_entities)
    if not entities_present:
        print("Not all entities were present in the generated summary...")
        print()
        print("Faulty summary: ", generated_text)
        print("Entities: ", named_entities)
    else:
        print("Output: ", generated_text)
        print("Entities: ", named_entities)

In [174]:
def run_ner_remove_unwanted_entities_test_case(test_case, prompt, model, tokenizer, ner_model, ner_tokenizer, threshold=0.99):
    print(f"Test Case: {test_case}")
    print(f"Model: {model.config.to_dict()['_name_or_path']}")
    print("----------------------------------------------")
    
    ner = pipeline('ner', model=ner_model, tokenizer=ner_tokenizer, aggregation_strategy="first")
    entities = ner(prompt)
    
    named_entities = []

    for entity in entities:
        if entity["score"] >= threshold and entity["entity_group"] == "PER":
            named_entities.append(entity["word"])
    named_entities = set(named_entities)
            
    input_ids = tokenizer.encode(prompt, return_tensors="pt")
    force_tokens = [tokenizer.encode(force_word, add_special_tokens=False) for force_word in named_entities]
    
    first_run = True
    entities_present = False
    bad_word_ids = None
    
    output = model.generate(
        input_ids,
        force_words_ids=force_tokens,
        num_beams=10,
        no_repeat_ngram_size=2,
        encoder_no_repeat_ngram_size=3,
        length_penalty=2,
        max_length=128,
        early_stopping=True
    )
    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
    
    new_entities = ner(generated_text)
    new_entity_words = set(entity["word"] for entity in new_entities if entity["score"] >= threshold and entity["entity_group"] == "PER")
    unwanted_entities = new_entity_words - set(named_entities)
    
    if unwanted_entities:
        for i in range(3):
            print(f"{i + 1}. iteration: Hallucinated Entities: {unwanted_entities}")
            original_unwanted_entities = unwanted_entities
            bad_word_ids = [tokenizer.encode(bad_word, add_special_tokens=False) for bad_word in unwanted_entities]

            output = model.generate(
                input_ids,
                force_words_ids=force_tokens,
                bad_words_ids=bad_word_ids,
                num_beams=10,
                no_repeat_ngram_size=2,
                encoder_no_repeat_ngram_size=3,
                length_penalty=2,
                max_length=128,
                early_stopping=True
            )
            generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
            new_entities = ner(generated_text)
            new_entity_words = set(entity["word"] for entity in new_entities if entity["score"] >= threshold and entity["entity_group"] == "PER")
            unwanted_entities = new_entity_words - set(named_entities)
            unwanted_entities = set(unwanted_entities).union(original_unwanted_entities)
            
            if all(entity in generated_text for entity in named_entities) and all(entity not in generated_text for entity in unwanted_entities):
                break
        print()
        print("Output: ", generated_text)
        print("Entities: ", named_entities)
        print("Hallucinated Entities: ", unwanted_entities)
    else:
        print("No hallucinated entities found...")
        print("----------------------------------------------")
        print("Output: ", generated_text)
        print("Entities: ", named_entities)

In [177]:
prompt = """
Ma van a trianoni békeszerződés 102. évfordulója, ez a nap 2010. óta Magyarországon a nemzeti összetartozás napja is egyben. Az évfordulóról a pártok és politikusok is megemlékeztek.

Fidesz – A kormánypárt Orbán Viktor soraival emlékezett meg az összetartozás napjáról: „Van egy nagy nemzet, amely messze túlnyúlik az országhatárokon. Ennek a lelki közösségnek a megőrzése olyan nehéz száz év után, mint amit megéltünk, mégiscsak sikerült. A nemzeti összetartozás gondolata, érzésvilága erős, talán erősebb, mint a korábbi évtizedekben bármikor is volt.”

KDNP – Közleményükben azt írják, hogy „történelmünk során sokszor váltunk nagyhatalmak játékszerévé, de mindig ragaszkodtunk függetlenségünkhöz, nyelvünkhöz, hitünkhöz, kultúránkhoz, tradícióinkhoz, amelyek biztosították nemzetünk fennmaradását a legnehezebb korszakokban is.” A párt kitért arra is, hogy szerintük a baloldali-liberális politika el akarta magától taszítani a határon túli magyarságot. Továbbá kiemelik, hogy Magyarország segítségnyújtását Kárpátaljának: „Az anyaország mindig segítő kezet nyújt a bajbajutottaknak, most különösképpen a kárpátaljai testvéreinknek”

Karácsony Gergely – „Trianon közös tragédiája a magyar nemzetnek, kitörölhetetlen. Számomra elsősorban édesanyám története, aki a második bécsi döntéssel visszacsatolt területen született” – írja le személyes történetét a főpolgármester. Felhívta a figyelmet, hogy Trianon kérdésében nincsenek oldalak, csak közös gyász. „Jó volna, hogy ha végre megtanulnánk a ”vagy„ helyett ”és„-t használni, és a nemzeti összetartozás napján megfogadni, Trianonból tanulva, hogy magyarokként egyek vagyunk.”
"""
test_case = "Historical Context"

In [178]:
run_ner_remove_unwanted_entities_test_case(test_case, prompt, model, tokenizer, ner_model, ner_tokenizer)

Test Case: Historical Context
Model: SZTAKI-HLT/Bert2Bert-HunSum-1
----------------------------------------------
1. iteration: Hallucinated Entities: {'Orbán Viktort', 'Kocsis Máté', 'Szatmáry Kristóf'}
2. iteration: Hallucinated Entities: {'Bobrovniczky Kristóf', 'Kókák Imre', 'Orbán Viktort', 'Kocsis Máté', 'Szatmáry Bianka', 'Szalay', '-', 'Szatmáry Kristóf'}
3. iteration: Hallucinated Entities: {'Bobrovniczky Kristóf', 'Kocsis Máté', 'Szatmáry Bianka', 'Szalay', 'Kókák Imre', 'Orbán Viktort', 'Molnár Zsolt', '-', 'Szatmáry Kristóf'}

Output:  A Orbán – Trianon megemlékezésen a Fidesz és a KDNP is megemlékezett. A Fidesz Orbán Viktor sorával emlékezett, a kormánypártok és az MSZP is. Videó! Videók! A képre kattintva galéria nyílik! FOTÓK : Molnár Ádám / Népszava. FNDATE : FACSONS / MOLNU / SZAVA, MOLSZF / FISZ / RNSA / SONON / FARARSS, SZEGOM, CS CSUU, KNNA / CSOMU.. VMSZ. SZÓS : SZNSS. Karácsony Gergely
Entities:  {'Karácsony Gergely', 'Orbán Viktor'}
Hallucinated Entities:  {'Bob

## Test Case 1: Maintaining Key Political Figures and Entities

In [142]:
prompt = """
Azért járt rendezvényről rendezvényre autóval Karácsony Gergely főpolgármester az autómentes hétvégén, mert egy térdsérülés miatt nem tudott biciklizni – válaszolta a Telex kérdéseire a Főpolgármesteri Hivatal Sajtókapcsolati Csoportja.

Hétfőn a Bors írt arról, hogy Karácsony mindenhová autóval ment a hétvégén, sőt azt írták, hogy a sofőrje még szabálytalanul is parkolt. Emellett a különböző rendezvényeken pont a kerékpározást és a tömegközlekedést népszerűsítette, még villamosokkal is fotózkodott.

A cikk miatt arról kérdeztük a Főpolgármesteri hivatalt, hogy valóban autózott-e, és arról, hogy valóban szabálytalanul parkolt-e.

„A főpolgármester a hétvégén öt különböző fővárosi programon vett részt. A bicikli használatát egy térdsérülés akadályozta. Ellentétben a propagandamédia állításával a szolgálati autó sofőrje nem parkolt a járdán, hanem egy társasház kapubeállójában állt meg a kiszállás idejére szabályosan”

– áll a hivatal válaszában.
Arra is rákérdeztünk, hogy a főpolgármester tudott-e volna tömegközlekedési eszközzel menni egyik rendezvényről a másikra, de erre nem kaptunk választ.
"""
test_case = "Maintaining Key Political Figures and Entities"

In [111]:
run_control_case(prompt, model, tokenizer)

Model: SZTAKI-HLT/Bert2Bert-HunSum-1
----------------------------------------------
Output:  A Gergely - buszsofõr nem állt meg egy baleset miatt, amikor a főpolgármester autómentes hétvégére igyekezett.


In [112]:
run_ner_test_case(test_case, prompt, model, tokenizer, ner_model, ner_tokenizer)

Test Case: Maintaining Key Political Figures and Entities
Model: SZTAKI-HLT/Bert2Bert-HunSum-1
----------------------------------------------
Output:  Karácsony Gergely autómentes hétvégére ment autóval, de a sofőr nem tudott megállni, mert a kocsija nem volt szabályos.
Entities:  ['Karácsony Gergely']


In [113]:
run_ner_remove_unwanted_entities_test_case(test_case, prompt, model, tokenizer, ner_model, ner_tokenizer)

Test Case: Maintaining Key Political Figures and Entities
Model: SZTAKI-HLT/Bert2Bert-HunSum-1
----------------------------------------------
No hallucinated entities found...
----------------------------------------------
Output:  Karácsony Gergely autómentes hétvégére ment autóval, de a sofőr nem tudott megállni, mert a kocsija nem volt szabályos.
Entities:  ['Karácsony Gergely']


In [114]:
prompt = """
Nem válaszolt Szijjártó Péter levelére a svéd külügyminiszter – árulta el az atv.hu-nak a magyar politikus.

A külgazdasági és külügyminiszter még szeptemberben írt kollégájának, melyben arra a magyar kormánnyal szemben kritikus hangokat megszólaltató svéd oktatóvideóra reagált, amely ugyan még 2019-ben készült, de a magyar kormánypárti politikusoknak csak most, 2023-ban jutott eszükbe felháborodni fel rajta. Szijjártó ezzel kapcsolatban figyelmeztette Tobias Billströmöt, nem szabad meglepődnie azon, ha a magyar képviselők sértésnek tekintik, hogy antidemokratikusnak vagy tekintélyelvűnek ítélik meg őket.

„Biztos vagyok benne, hogy érti a két jelenség közötti ellentmondást: önök sürgetik országgyűlési képviselőinket, hogy ratifikálják az önök NATO-csatlakozását, miközben továbbra is azzal vádolják őket, hogy lerombolták a magyarországi demokráciát. (…) Ez az ellentmondás most még inkább kiéleződött, és cseppet sem segíti az ön folyamatos követelésének teljesítését” – írta Szijjártó.

A svéd NATO-csatlakozás ratifikációja már több mint másfél éve húzódik, úgy, hogy időközben a finn NATO-csatlakozást megszavazta a parlament, a svéd javaslatot pedig már Erdogan török elnök is tovább küldte az ankarai nemzetgyűlésnek.

Szijjártó a lapnak arról beszélt, nem lepődött meg a válasz elmaradásán.

„Jól látszik a svédek hozzáállása, hogy nem is akarnak reagálni, ami szerintem baj” – fogalmazott a politikus, aki szerint
"""
test_case = "Maintaining Key Political Figures and Entities"

In [115]:
run_control_case(prompt, model, tokenizer)

Model: SZTAKI-HLT/Bert2Bert-HunSum-1
----------------------------------------------
Output:  Szijjártó svéd kollégájának írt levelére reagált Szijjártó Péter, de nem válaszolt.


In [116]:
run_ner_test_case(test_case, prompt, model, tokenizer, ner_model, ner_tokenizer)

Test Case: Maintaining Key Political Figures and Entities
Model: SZTAKI-HLT/Bert2Bert-HunSum-1
----------------------------------------------
Output:  Szijjártó külügyminiszter nem reagált Szijjártó Péter levelere, melyben a svéd diplomata arra figyelmeztette a magyar kormányt, hogy ne szavazza meg a NATO - csatlakozási szerzõdést, mert ezzel lerombolják az alkotmányos rendet.
Entities:  ['Szijjártó Péter']


In [117]:
run_ner_remove_unwanted_entities_test_case(test_case, prompt, model, tokenizer, ner_model, ner_tokenizer)

Test Case: Maintaining Key Political Figures and Entities
Model: SZTAKI-HLT/Bert2Bert-HunSum-1
----------------------------------------------
No hallucinated entities found...
----------------------------------------------
Output:  Szijjártó külügyminiszter nem reagált Szijjártó Péter levelere, melyben a svéd diplomata arra figyelmeztette a magyar kormányt, hogy ne szavazza meg a NATO - csatlakozási szerzõdést, mert ezzel lerombolják az alkotmányos rendet.
Entities:  ['Szijjártó Péter']


## Test Case 3: Sports Reporting

In [131]:
prompt = """
Halmozza a győzelmeket és a rekordokat Max Verstappen 2023-ban, ám ez a pénztárcájának várhatóan fájni fog.
A szezonbeli 17. győzelmével Max Verstappen Brazíliában 524-re növelte az idén szerzett pontjainak számát. Ezzel már most az egy szezonon belül legtöbb pontot szerző versenyzője a Forma–1 történetének. És még hátravan két verseny, melyeken további 52 pont gyűjthető.

Az elképesztő fölényének azonban van hátulütője is. Ahogy arra az ausztrál Speedcafe rámutatott, a háromszoros világbajnoknak az idei sikerei árát egy várhatóan rekord összegű szuperlicenc formájában kell majd megfizetnie, melynek értéke minden valószínűség szerint meg fogja haladni az 1 millió eurót.

Bár Verstappen idei végső pontszámát még nem ismerjük, és az FIA még nem tette közzé a 2024-es szuperlicenc megváltásának számait, a tavalyi adatok alapján már most biztosnak tűnik hogy a holland többet fog fizetni, mint korábban bármikor.
"""
test_case = "Sports Reporting"

In [132]:
run_control_case(prompt, model, tokenizer)

Model: SZTAKI-HLT/Bert2Bert-HunSum-1
----------------------------------------------
Output:  A FIA közzétette a 2021 - es versenysorozat rekordnyereményeit.


In [133]:
run_ner_test_case(test_case, prompt, model, tokenizer, ner_model, ner_tokenizer)

Test Case: Sports Reporting
Model: SZTAKI-HLT/Bert2Bert-HunSum-1
----------------------------------------------
Output:  A Verstapen F1 - es pályafutása során eddig szerzett pontjai számát tekintve a holland klasszis idén is a csúcsra törhet. Az FIA azonban nem adta meg a módját, hogy a rekordot megdöntse. A 2024 – es szezon ugyanis a Red Bullé lesz. De vajon mi lesz a bajnokokkal 2022 - ben? És mi vár a háromszoros bajnokra? A Speeddafe összeszedte a legfontosabb eredményeket. És azt is elárulta, melyikük mennyit fog fizetni. Videó! FOTÓ! Ftorony! ( FECA ) VLT Max Verstappen
Entities:  {'Max Verstappen'}


In [134]:
run_ner_remove_unwanted_entities_test_case(test_case, prompt, model, tokenizer, ner_model, ner_tokenizer)

Test Case: Sports Reporting
Model: SZTAKI-HLT/Bert2Bert-HunSum-1
----------------------------------------------
No hallucinated entities found...
----------------------------------------------
Output:  A Verstapen F1 - es pályafutása során eddig szerzett pontjai számát tekintve a holland klasszis idén is a csúcsra törhet. Az FIA azonban nem adta meg a módját, hogy a rekordot megdöntse. A 2024 – es szezon ugyanis a Red Bullé lesz. De vajon mi lesz a bajnokokkal 2022 - ben? És mi vár a háromszoros bajnokra? A Speeddafe összeszedte a legfontosabb eredményeket. És azt is elárulta, melyikük mennyit fog fizetni. Videó! FOTÓ! Ftorony! ( FECA ) VLT Max Verstappen
Entities:  {'Max Verstappen'}


In [143]:
prompt = """
A Red Bull csapatvezetője, Christian Horner úgy véli, Daniel Ricciardo „nagyon rossz tanácsokat kapott”, amikor 2018-ban úgy döntött, elhagyja a Red Bullt, majd hozzátette, érzése szerint az ausztrál már belátja, hibázott.
Miután hosszú évek után távozott a bikásoktól, Ricciardo többször utalt rá, hogy el akart szakadni nevelőistállójától, valamint kezdte úgy érezni, a csapat egyre inkább Max Verstappenre épít, aki utolsó szezonjukban már egyre jobban a „fejére nőtt”. Először a Renault-hoz igazolt, ahol vezérpilótaként számítottak rá, kalandja viszont csak két évig tartott.

Ez alatt az időszak alatt jól teljesített, több dobogós helyezést is szerzett, a győzelem és a nagy fejlődés viszont nem jött össze. Innen igazolt a McLarenhez, ahol két év alatt is nehezen vette fel a ritmust és 2022-ben, idő előtt távozni kényszerült. A Red Bull ekkor fogadta vissza tartalékosnak.

Idén év közben aztán az Alpha Tauri versenyzője lett, javuló eredményeit látva pedig egyre többen pletykálnak arról, lehet számára visszaút a Red Bullhoz is. Ezt a csapatfőnök, Christian Horner sem zárta ki, aki nemrég Eff Won With DRS podcastben beszélt az ausztrálról, megjegyezve, érzése szerint Ricciardo 2018-as távozása előtt „rossz tanácsokat kapott”.
"""
test_case = "Sports Reporting"

In [144]:
run_control_case(prompt, model, tokenizer)

Model: SZTAKI-HLT/Bert2Bert-HunSum-1
----------------------------------------------
Output:  Christ Red Bull - csapatfőnök állítja, hogy az ausztrál hibázott, amikor otthagyta az istállót, de azt is elmondta, miért.


In [145]:
run_ner_test_case(test_case, prompt, model, tokenizer, ner_model, ner_tokenizer)

Test Case: Sports Reporting
Model: SZTAKI-HLT/Bert2Bert-HunSum-1
----------------------------------------------
Not all entities were present in the generated summary...

Faulty summary:  A Az Red Bullnál a csapatvezető, Christopher Horner szerint Daniel Riccirdo rossz tanácsokat adott neki 2018 - tól, de a Max Versthenhez való távozása után is maradt még ideje. A csapatfőnök szerint Christian Streamnek is van véleménye.. De mi lesz a Ricciartól távozó Danieltel? Eff St. George szerint.?.!!. (. ). Itt a RedBank. Videó. Spoiler : spoirtveszély. Az F1 - bõl.. Voice. com. FOT
Entities:  {'Daniel Ricciardo', 'Eff Won With', 'Max Verstappenre', 'Ricciardo', 'Christian Horner'}


In [152]:
run_ner_remove_unwanted_entities_test_case(test_case, prompt, model, tokenizer, ner_model, ner_tokenizer)

Test Case: Sports Reporting
Model: SZTAKI-HLT/Bert2Bert-HunSum-1
----------------------------------------------
SAD: {'Daniel Riccirdo', 'Max Versthenhez', 'Christopher Horner', 'Christian Streamnek'}
SAD: {'Max Verstley - t', 'Max Versthenhez', 'Daniel Riccirdo', 'Christopher Hornernek', 'Christopher Horner', 'Christian', 'Christian Streamnek'}
SAD: {'Max Verstley - t', 'Max Versthenhez', 'Max Verstppen - korszakból', 'Rossian Hornernek', 'Daniel Riccirdo', 'Christopher Hornert', 'Christopher Hornernek', 'Christopher Horner', 'Christian', 'Christian Streamnek'}
Output:  A Az Red Bullnál a csapatvezető, Christopher Hornerhez hasonlóan a Max Verst Vettelt irányító Danny Brawn is azt mondta, hogy Daniel Riccirdo rossz tanácsokat kap, amikor elhagyja a csapatot. A csapatfőnök szerint azonban a Ricciart kritizáló ausztrálnak igaza van. Az ausztrál viszont azt mondja, a RedLarent nem szabad leírni.. De mi lesz a csapattal?. Effee - interjú.??!!.!? Spíling! ( Speeds to the Speds ). Itt a cik

# Length Constraint

In [3]:
def run_control_case(prompt, model, tokenizer):
    print(f"Model: {model.config.to_dict()['_name_or_path']}")
    print("----------------------------------------------")

    input_ids = tokenizer.encode(prompt, return_tensors="pt")
    output = model.generate(
        input_ids,
        num_beams=10,
        no_repeat_ngram_size=2,
        encoder_no_repeat_ngram_size=3
    )
    
    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
    print("Output: ", generated_text)

In [139]:
def run_length_test_case(test_case, prompt, model, tokenizer, min_length=30, max_length=128, length_penalty=2.0, early_stopping=True):
    print(f"Test Case: {test_case}")
    print(f"Model: {model.config.to_dict()['_name_or_path']}")
    print("----------------------------------------------")

    input_ids = tokenizer.encode(prompt, return_tensors="pt")
    
    output = model.generate(
        input_ids,
        num_beams=10,
        length_penalty=length_penalty,
        min_length=min_length,
        max_length=max_length,
        early_stopping=early_stopping,
        no_repeat_ngram_size=2
    )
    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
    print("Output: ", generated_text)

## Test Case 1: Short Input Text

In [148]:
prompt = """
A gyógypedagógusok és asszisztensek mellett dajka, takarító, karbantartó is felmondott, mert nem tudták elviselni a bizonytalan pénzügyi helyzetet, hogy nyár óta a bérüknek csak egy részét kapták meg.

Arról már többször is beszámoltunk a Telexen, hogy a Magyarországi Evangéliumi Testvérközösség (MET) intézményei, köztük az egyház és alapítványa által fenntartott iskolák is nehéz pénzügyi helyzetben vannak, szerintük azért, mert a kormány nem utalja ki számukra a nekik járó állami támogatásokat.
"""
test_case = "Short Input Text"

In [149]:
run_control_case(prompt, model, tokenizer)

Model: SZTAKI-HLT/Bert2Bert-HunSum-1
----------------------------------------------
Output:  A,, szeretetotthonok " intézményei közül sokan felmondtak, mert fizetésük egy részét még mindig nem kapták meg a bérükbõl.


In [150]:
run_length_test_case(test_case, prompt, model, tokenizer, min_length=30, max_length=128, length_penalty=2.0, early_stopping=True)

Test Case: Short Input Text
Model: SZTAKI-HLT/Bert2Bert-HunSum-1
----------------------------------------------
Output:  A a Magyarországi Evangéliumi Testvérközösség ( MET ) intézményei nem kapták meg a bérük egy részét, mert a kormány nem utalja ki nekik a nekik járó állami támogatást.


## Test Case 2: Long Input Text

In [151]:
prompt = """
A pluszterületet a környező, önkormányzati tulajdonú ingatlanok rovására építették be. A Pénzügyminisztérium válaszul átad 25 négyzetmétert a Szentháromság tér közelében, de az önkormányzat tulajdonvesztése ezzel továbbra is 387 négyzetméter marad – írja a lap a kerület csütörtöki testületi ülésére készült előterjesztésének alapján.

„A dolog úgy derült ki, hogy kértek egy telekrendezést, amire a kerület felülvizsgálatot kezdeményezett, és kiderült a jelentős túlépítés. Az ingatlan értékbecslése megtörtént, a magyar állam 207 millió forintot fizet a közterületekért” – idézi a Népszava az I. kerületi polgármestert.

A kormány 2017-ben döntött úgy, hogy ismét beköltözteti a Várba a Pénzügyminisztériumot. 2020-ban kezdődött az építkezés, akkor 54 milliárdot szántak rá, az idén nyárra már 100 milliárd felett járó költségekhez augusztusban újabb 4,4 milliárdot írtak hozzá. A pénz bútorokra, audiovizuális eszközökre, belsőépítészeti munkákra, piperekiegészítőkre és teakonyhákra kell. A feladatot két NER-kötelékbe tartozó cégre bízták, írja a lap.
"""
test_case = "Long Input Text"

In [152]:
run_control_case(prompt, model, tokenizer)

Model: SZTAKI-HLT/Bert2Bert-HunSum-1
----------------------------------------------
Output:  A II. kerületi önkormányzat visszakapta a Budai Várba beköltöztetett Pénzügyminisztérium ( PM ) pluszterületét, amelyet a várba költöztetett a kormány.


In [153]:
run_length_test_case(test_case, prompt, model, tokenizer, min_length=30, max_length=128, length_penalty=2.0, early_stopping=True)

Test Case: Long Input Text
Model: SZTAKI-HLT/Bert2Bert-HunSum-1
----------------------------------------------
Output:  A a Szentháromság térnél, a Várba költözik a Pénzügyminisztérium – írja a Népszava az I. kerületi önkormányzat előterjesztésére hivatkozva. A lap szerint a kormány 207 millió forintot fizet ki a kerületnek.


# Repetition Control

In [49]:
def run_control_case(prompt, model, tokenizer):
    print(f"Model: {model.config.to_dict()['_name_or_path']}")
    print("----------------------------------------------")

    input_ids = tokenizer.encode(prompt, return_tensors="pt")
    output = model.generate(
        input_ids,
        num_beams=10,
        length_penalty=2,
        max_length=128,
        early_stopping=True,
        encoder_no_repeat_ngram_size=1
    )
    
    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
    print("Output: ", generated_text)

In [64]:
def run_repetition_test_case(test_case, prompt, model, tokenizer, no_repeat_ngram_size=2, encoder_no_repeat_ngram_size=3, repetition_penalty=2.0):
    print(f"Test Case: {test_case}")
    print(f"Model1: {model.config.to_dict()['_name_or_path']}")
    print("----------------------------------------------")

    input_ids = tokenizer.encode(prompt, return_tensors="pt")
    
    output = model.generate(
        input_ids,
        num_beams=10,
        no_repeat_ngram_size=no_repeat_ngram_size,
        encoder_no_repeat_ngram_size=encoder_no_repeat_ngram_size,
        repetition_penalty=repetition_penalty,
        length_penalty=2,
        max_length=128,
        early_stopping=True
    )
    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
    print("Output: ", generated_text)

## Test Case 1: Redundant Information

In [78]:
prompt = """
Az a vezetés, ami Brüsszelben van, egy globalista elit megbízását teljesíti - jelentette ki a miniszterelnök pénteken a Kossuth rádió Jó reggelt, Magyarország! című műsorában.

Orbán Viktor kiemelte: a brüsszeli vezetés nemcsak nem azt csinálja, amit a magyarok szeretnének, hanem azt sem, amit általában az európai emberek. Az emberek nem akarnak migrációt, háborút, békétlenséget, jól megtervezett zöld átmenetet akarnak, és nem olyat, ami tönkreteszi az iparukat - tette hozzá.

A kormányfő rámutatott:
 a brüsszeli vezetést foglyul ejtette egy globalista elit, pénzügyi csoportok, nagy gazdasági erőcsoportok, és a brüsszeliek döntéseit ezek érdekei motiválják, nem pedig a magyar, német, francia vagy az olasz emberek érdeke.

Ezért kell változást elérni - jelentette ki a miniszterelnök
"""
test_case = "Redundant Information"

In [79]:
run_control_case(prompt, model, tokenizer)

Model: SZTAKI-HLT/Bert2Bert-HunSum-1
----------------------------------------------
Output:  Nem " kormányátalakítás " kezdõdött Budapest belvárosában péntek délelõtt ; ez lesz Európa új vezetõinek elsõ találkozója ; ez lesz Európa új vezetõinek elsõ találkozója ; õrizni fogják Budapest látképét ; õrizni fogják Budapest látképét ; õrizni fogják Budapest látképét ; õrizni fogják Budapest látképét ; õrizni fogják Budapest látképét ; õrizni fogják Budapest látképét ; õrizni fogják Budapest látképét is ; õrizni fogják Budapest látképét is ; õrizni fogják


In [80]:
run_repetition_test_case(test_case, prompt, model, tokenizer, no_repeat_ngram_size=2, encoder_no_repeat_ngram_size=3, repetition_penalty=2.0)

Test Case: Redundant Information
Model1: SZTAKI-HLT/Bert2Bert-HunSum-1
----------------------------------------------
Output:  A Viktor miniszterelnök péntek reggeli rádióinterjújában arról beszélt, hogy a brüsszeli elitnek egy globális elit megbízásából kell cselekednie.


In [81]:
prompt = """
Idegenrendészeti szigorításra van szükség a növekvő migrációs nyomás miatt - jelentette ki a miniszterelnök péntek reggel a Kossuth rádió Jó reggelt, Magyarország! című műsorában.

Orbán Viktor az interjúban azt mondta, a jelenleg hatályos magyar szabályozás jól kezelte a 2015-ös migrációs válságot, a Brüsszellel vívott csatáknak jó eszköze volt, de a migrációs nyomás nő, ezért szigorítani kell a jogszabályon.
"""
test_case = "Redundant Information"

In [82]:
run_control_case(prompt, model, tokenizer)

Model: SZTAKI-HLT/Bert2Bert-HunSum-1
----------------------------------------------
Output:  A A belpolitikában is szigorúbb fellépést rendelt el Angela Merkel német kancellár és Charles Michel francia államfõ is Magyarországra érkezendõ migránshullámmal szemben : szigorúbb ellenõrzéseket rendeltek el és szigorúbb határellenõrzéseket rendeltek el minden migránstáborban és menekülttáborban – hangzott el egy szerda reggeli tévéinterjúban – tudta meg reggeli hírösszefoglalóban tudósítóinktól egy szerda reggeli rádiós híradásban vagy egy reggeli rádiós hírmûsorban ( reggeli hírmûsorban vagy hírmûsorban nem is hallottunk ilyen jelentõsebb beavatkozást ( ez nem is igaz ) ) )


In [83]:
run_repetition_test_case(test_case, prompt, model, tokenizer, no_repeat_ngram_size=3, encoder_no_repeat_ngram_size=3, repetition_penalty=2.0)

Test Case: Redundant Information
Model1: SZTAKI-HLT/Bert2Bert-HunSum-1
----------------------------------------------
Output:  A Viktor miniszterelnök által péntek reggel elmondott rádióinterjú szerint szigorúbb migrációs nyomásra van szükség, mert a mostani migrációs nyomás nem volt jó.


# Lemmatization

In [1]:
def lemmatize(sentences, nlp):
    lemmatized = []
    if isinstance(sentences, str):
        sentences = [sentences]

    for sentence in sentences:
        result = nlp(sentence)
        for res in result:
            lemmatized.append(res)

    return lemmatized[0] if len(lemmatized) == 1 else lemmatized

In [14]:
def run_omit_word_lemma_test_case(test_case, prompt, omit_words, model, tokenizer, nlp):
    print(f"Test Case: {test_case}")
    print(f"Model: {model.config.to_dict()['_name_or_path']}")
    print("----------------------------------------------")
    
    input_ids = tokenizer.encode(prompt, return_tensors="pt")
    omit_tokens = []
    for omit_word in omit_words:
        token = tokenizer.encode(omit_word, add_special_tokens=False)
        omit_tokens.append(token)
        
    output = model.generate(
        input_ids,
        bad_words_ids=omit_tokens,
        num_beams=10,
        no_repeat_ngram_size=2,
        encoder_no_repeat_ngram_size=3,
        length_penalty=2,
        min_length=30,
        max_length=128,
        early_stopping=True
    )
    
    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
    lemmatized_text = lemmatize(generated_text, nlp)
    lemmatized_omit_words = lemmatize(omit_words, nlp)
    new_run = False
    
    for omit_word in lemmatized_omit_words:
        for text in lemmatized_text:
            if omit_word.lemma_ == text.lemma_:
                new_run = True
                omit_words.append(text.text)
                
    while new_run:
        print(f"Retrying...")
        print(generated_text)
        omit_words = list(set(omit_words))
        omit_tokens = []
        for omit_word in omit_words:
            token = tokenizer.encode(omit_word, add_special_tokens=False)
            omit_tokens.append(token)
            
        print(f"New Omit words: {omit_words}")
        input_ids = tokenizer.encode(generated_text, return_tensors="pt")
        output = model.generate(
            input_ids,
            bad_words_ids=omit_tokens,
            num_beams=10,
            no_repeat_ngram_size=2,
            encoder_no_repeat_ngram_size=3,
            length_penalty=2,
            max_length=128,
            early_stopping=True
        )
        generated_text = tokenizer.decode(output[0], skip_special_tokens=True)            
        if all(omit_word not in generated_text for omit_word in omit_words):
            new_run = False
            break
        
        lemmatized_text = lemmatize(generated_text, nlp)
        lemmatized_omit_words = lemmatize(omit_words, nlp)
        for omit_word in lemmatized_omit_words:
            for text in lemmatized_text:
                if omit_word.lemma_ == text.lemma_:
                    new_run = True
                    omit_words.append(text.text)
    print()
    print("Output: ", generated_text)
    print("Final Omits: ", omit_words)

In [17]:
prompt = """
Novembertől a nagyközönség számára is elérhetővé vált a Credential Manager nevezetű funkció, amelynek köszönhetően a korábbinál sokkal biztonságosabb platformmá válhat az Android – legalábbis a Google szerint.
Ahogy arról korábban már az Index is beszámolt, az idén októberben megtartott Google I/O-n rengeteg mindenről szó esett. A Google amellett, hogy bemutatta legújabb telefonjait, a Google Pixel 8-at és a Pixel 8 Prót, illetve új okosóráját, a Google Pixel Watch 2-t, egyéb témákra is kitért – például a biztonságra.
"""
bad_words = ["Google", "biztonságosabbá"]
test_case = "Negation"

In [12]:
run_control_case(prompt, model, tokenizer)

Model: SZTAKI-HLT/mT5-base-HunSum-1
----------------------------------------------
Output:  A Google szerint az Android biztonságosabbá válhat, mint az eddigi platformok.


In [18]:
run_omit_word_lemma_test_case(test_case, prompt, bad_words, model, tokenizer, nlp)

Test Case: Negation
Model: SZTAKI-HLT/mT5-base-HunSum-1
----------------------------------------------
Retrying...
Többek között arról is beszélt a keresőóriás, milyen biztonsági funkciót kapott az Android, és hogy mennyire biztonságos az operációs rendszer.
New Omit words: ['Google', 'biztonságosabbá', 'biztonságos']

Output:  Az Android fejlesztői a közösségi oldalukon tették közzé, hogy milyen funkciók vannak az Android operációs rendszerben.
Final Omits:  ['Google', 'biztonságosabbá', 'biztonságos']


# Evaluation

In [28]:
huspacy.download("hu_core_news_lg")
nlp = spacy.load("hu_core_news_lg")

In [29]:
def preprocess_text(raw_text, nlp, remove_punctuation=False, lower_case=True, remove_stopwords=False, lemmatize=True):
    if lower_case:
        raw_text = raw_text.lower()
        
    if remove_punctuation:
        raw_text = raw_text.translate(str.maketrans('', '', string.punctuation))

    doc = nlp(raw_text)

    tokens = []

    for token in doc:
        if lemmatize:
            token_text = token.lemma_
        else:
            token_text = token.text

        if remove_stopwords:
            if token_text.lower() not in nlp.Defaults.stop_words and not token.is_punct:
                tokens.append(token_text)
        else:
            if not token.is_punct:
                tokens.append(token_text)
    
    preprocessed_text = ' '.join(tokens)
    
    return preprocessed_text

def run_rouge(nlp, control, generated, reference):
    reference_processed = preprocess_text(reference, nlp, remove_punctuation=True, lower_case=True, remove_stopwords=False)
    generated_processed = preprocess_text(generated, nlp, remove_punctuation=True, lower_case=True, remove_stopwords=False)
    
    rouge = Rouge()
    control_processed = preprocess_text(control, nlp, remove_punctuation=True, lower_case=True, remove_stopwords=False)
    control_scores = rouge.get_scores(control_processed, reference_processed)
    
    generated_scores = rouge.get_scores(generated_processed, reference_processed)
    
    return control_scores[0], generated_scores[0]

def run_bleu(control, generated, reference):
    reference_processed = preprocess_text(reference, nlp, remove_punctuation=False, lower_case=True, remove_stopwords=False)
    generated_processed = preprocess_text(generated, nlp, remove_punctuation=False, lower_case=True, remove_stopwords=False)
    control_processed = preprocess_text(control, nlp, remove_punctuation=False, lower_case=True, remove_stopwords=False)
    
    reference_processed = [reference_processed.split()]
    generated_processed = generated_processed.split()
    control_processed = control_processed.split()
    
    control_score = sentence_bleu(reference_processed, control_processed, smoothing_function=SmoothingFunction().method2)
    generated_score = sentence_bleu(reference_processed, generated_processed, smoothing_function=SmoothingFunction().method2)
    
    return control_score, generated_score

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\nyist\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\nyist\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [30]:
def run_metrics(nlp, control, generated, reference):
    control_rouge_scores, generated_rouge_scores = run_rouge(nlp, control, generated, reference)
    control_bleu_score, generated_bleu_score = run_bleu(control, generated, reference)

    results = {
        'Metric': ['ROUGE-1', 'ROUGE-2', 'ROUGE-L', 'BLEU'],
        'Control': [control_rouge_scores['rouge-1']['f'], control_rouge_scores['rouge-2']['f'], control_rouge_scores['rouge-l']['f'], control_bleu_score],
        'Generated': [generated_rouge_scores['rouge-1']['f'], generated_rouge_scores['rouge-2']['f'], generated_rouge_scores['rouge-l']['f'], generated_bleu_score]
    }

    df = pd.DataFrame(results)
    return df

In [36]:
def run_metrics_multiple(nlp, controls, generateds, references):
    total_scores = {
        'ROUGE-1': {'Control': 0, 'Generated': 0},
        'ROUGE-2': {'Control': 0, 'Generated': 0},
        'ROUGE-L': {'Control': 0, 'Generated': 0},
        'BLEU': {'Control': 0, 'Generated': 0}
    }
    
    for control, generated, reference in zip(controls, generateds, references):
        if control and generated and reference:
            control_rouge_scores, generated_rouge_scores = run_rouge(nlp, control, generated, reference)
            control_bleu_score, generated_bleu_score = run_bleu(control, generated, reference)

            total_scores['ROUGE-1']['Control'] += control_rouge_scores['rouge-1']['f']
            total_scores['ROUGE-1']['Generated'] += generated_rouge_scores['rouge-1']['f']
            total_scores['ROUGE-2']['Control'] += control_rouge_scores['rouge-2']['f']
            total_scores['ROUGE-2']['Generated'] += generated_rouge_scores['rouge-2']['f']
            total_scores['ROUGE-L']['Control'] += control_rouge_scores['rouge-l']['f']
            total_scores['ROUGE-L']['Generated'] += generated_rouge_scores['rouge-l']['f']
            total_scores['BLEU']['Control'] += control_bleu_score
            total_scores['BLEU']['Generated'] += generated_bleu_score

    # Calculate the average scores
    num_cases = len(controls)
    for metric in total_scores:
        for key in total_scores[metric]:
            total_scores[metric][key] /= num_cases

    results = {
        'Metric': list(total_scores.keys()),
        'Control': [total_scores[metric]['Control'] for metric in total_scores],
        'Generated': [total_scores[metric]['Generated'] for metric in total_scores]
    }

    return pd.DataFrame(results)

In [103]:
control = "It is a guide to action which ensures that the military always obeys the commands of the party"
generated = 'It is a guide to action which ensures that the military will forever heed Party commands'
reference = 'It is a guide to action which ensures that the military will forever heed Party commands'

df = run_metrics(nlp, control, generated, reference)
df

,Metric,Control,Generated
0,ROUGE-1,0.812500,1.0
1,ROUGE-2,0.625000,1.0
2,ROUGE-L,0.750000,1.0
3,BLEU,0.618182,1.0
